In [1]:
import ccxt
import time

# API Keys (Replace with actual keys)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Connect to exchanges
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})


In [ ]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
TRADING_FEE = 0.001  # 0.1% trading fee per trade

def get_tradable_tokens(exchange, exchange_name):
    """Fetch tradable spot tokens and filter out low-volume pairs"""
    try:
        markets = exchange.load_markets()
        tradable_tokens = []

        for symbol in markets:
            if "/USDT" in symbol and ":" not in symbol:  # Spot trading pairs
                market_data = exchange.fetch_ticker(symbol)
                volume = market_data.get('quoteVolume', 0) or market_data.get('baseVolume', 0)
                if volume and volume >= MIN_VOLUME:
                    tradable_tokens.append(symbol)

        print(f"✅ Found {len(tradable_tokens)} spot tokens on {exchange_name}")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from {exchange_name}: {e}")
        return []

def get_price(exchange, symbol):
    """Fetch latest price for a given token if the market exists"""
    try:
        if symbol in exchange.load_markets():
            ticker = exchange.fetch_ticker(symbol)
            return float(ticker.get("last", ticker.get("lastPrice", None)))
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol}: {e}")
        return None

def check_inter_exchange_arbitrage(symbol):
    """Checks for arbitrage opportunities between Bybit and Bitget for Spot Trading"""
    bybit_price = get_price(bybit, symbol)
    bitget_price = get_price(bitget, symbol)

    if not bybit_price or not bitget_price:
        return

    spread_buy_bybit_sell_bitget = (bitget_price - bybit_price) / bybit_price
    spread_buy_bitget_sell_bybit = (bybit_price - bitget_price) / bitget_price

    net_profit_bybit_to_bitget = spread_buy_bybit_sell_bitget - (TRADING_FEE * 2)
    net_profit_bitget_to_bybit = spread_buy_bitget_sell_bybit - (TRADING_FEE * 2)

    if net_profit_bybit_to_bitget >= SPREAD_THRESHOLD:
        print(f"🔍 Inter-exchange Arbitrage Detected: Buy on Bybit Spot, Sell on Bitget Spot for {symbol}")
        print(f"    ✅ Bybit Spot Price: {bybit_price:.5f} USDT")
        print(f"    ✅ Bitget Spot Price: {bitget_price:.5f} USDT")
        print(f"    ✅ Spread: {spread_buy_bybit_sell_bitget:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit_bybit_to_bitget:.4%}")
        print("-" * 50)

    if net_profit_bitget_to_bybit >= SPREAD_THRESHOLD:
        print(f"🔍 Inter-exchange Arbitrage Detected: Buy on Bitget Spot, Sell on Bybit Spot for {symbol}")
        print(f"    ✅ Bitget Spot Price: {bitget_price:.5f} USDT")
        print(f"    ✅ Bybit Spot Price: {bybit_price:.5f} USDT")
        print(f"    ✅ Spread: {spread_buy_bitget_sell_bybit:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit_bitget_to_bybit:.4%}")
        print("-" * 50)

while True:
    try:
        bybit_tokens = get_tradable_tokens(bybit, "Bybit")
        bitget_tokens = get_tradable_tokens(bitget, "Bitget")

        common_spot_tokens = list(set(bybit_tokens) & set(bitget_tokens))

        if not common_spot_tokens:
            print("❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...")
            time.sleep(10)
            continue

        print("🔄 Checking for arbitrage opportunities...")

        for token in common_spot_tokens:
            check_inter_exchange_arbitrage(token)

        print("✅ Completed this cycle. Waiting for next check...")
        time.sleep(CHECK_INTERVAL)

    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)

❌ Error fetching tradable tokens from Bybit: bybit GET https://api.bybit.com/v5/market/tickers?symbol=ANIMEUSDT&category=spot
❌ Error fetching tradable tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...
❌ Error fetching tradable tokens from Bybit: bybit GET https://api.bybit.com/v5/market/tickers?symbol=BTCUSDT&category=spot
❌ Error fetching tradable tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...
❌ Error fetching tradable tokens from Bybit: bybit GET https://api.bybit.com/v5/market/tickers?symbol=BTCUSDT&category=spot
❌ Error fetching tradable tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common tradable tokens found between Bybit and Bitget. Retrying in 10 seconds...
❌ Error fetching tradable tokens from Bybit: bybit GE